# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [83]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dangunnarsson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dangunnarsson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dangunnarsson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [37]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [39]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [40]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [84]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline_rf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [43]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [86]:
pipeline_rf.fit(X_train, Y_train)

/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [46]:
Y_pred = pipeline.predict(X_test)
    
Y_pred_data_frame = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('Feature: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_data_frame[column]))

Feature: request

              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90      4338
         1.0       0.46      0.13      0.21       904

    accuracy                           0.82      5242
   macro avg       0.65      0.55      0.55      5242
weighted avg       0.78      0.82      0.78      5242

Feature: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5221
         1.0       0.00      0.00      0.00        21

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      5242
weighted avg       0.99      1.00      0.99      5242

Feature: aid_related

              precision    recall  f1-score   support

         0.0       0.59      0.88      0.71      3048
         1.0       0.47      0.15      0.22      2194

    accuracy                           0.57      5242
   macro avg       0.53      0.51      0.46      5242
weighted avg      

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      5129
         1.0       0.00      0.00      0.00       113

    accuracy                           0.98      5242
   macro avg       0.49      0.50      0.49      5242
weighted avg       0.96      0.98      0.97      5242

Feature: other_weather

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      4962
         1.0       0.00      0.00      0.00       280

    accuracy                           0.95      5242
   macro avg       0.47      0.50      0.49      5242
weighted avg       0.90      0.95      0.92      5242

Feature: direct_report

              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89      4222
         1.0       0.46      0.10      0.16      1020

    accuracy                           0.80      5242
   macro avg       0.64      0.53      0.52      5242
weighted avg       0.75   

/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [80]:
overall_accuracy = (Y_pred == Y_test).mean().mean()
print(overall_accuracy)

0.9311658581784489


In [87]:
Y_pred_rf = pipeline_rf.predict(X_test)
    
Y_pred_rf_data_frame = pd.DataFrame(Y_pred_rf, columns = Y_test.columns)
for column in Y_test.columns:
    print('Feature: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_rf_data_frame[column]))

Feature: request

              precision    recall  f1-score   support

         0.0       0.84      0.98      0.90      4338
         1.0       0.46      0.07      0.12       904

    accuracy                           0.83      5242
   macro avg       0.65      0.53      0.51      5242
weighted avg       0.77      0.83      0.77      5242

Feature: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5221
         1.0       0.00      0.00      0.00        21

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      5242
weighted avg       0.99      1.00      0.99      5242

Feature: aid_related

              precision    recall  f1-score   support

         0.0       0.58      0.82      0.68      3048
         1.0       0.43      0.18      0.26      2194

    accuracy                           0.56      5242
   macro avg       0.51      0.50      0.47      5242
weighted avg      

/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      5166
         1.0       1.00      0.01      0.03        76

    accuracy                           0.99      5242
   macro avg       0.99      0.51      0.51      5242
weighted avg       0.99      0.99      0.98      5242

Feature: money

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      5120
         1.0       0.00      0.00      0.00       122

    accuracy                           0.98      5242
   macro avg       0.49      0.50      0.49      5242
weighted avg       0.95      0.98      0.96      5242

Feature: missing_people

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      5184
         1.0       0.00      0.00      0.00        58

    accuracy                           0.99      5242
   macro avg       0.49      0.50      0.50      5242
weighted avg       0.98      0.99

In [88]:
overall_accuracy = (Y_pred_rf == Y_test).mean().mean()
print(overall_accuracy)

0.9295252629857744


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
    'features__text_pipeline__tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, Y_train)
cv_y_pred = cv.predict(X_test)
cv_Y_pred_data_frame = pd.DataFrame(cv_y_pred, columns = Y_test.columns)

/Users/dangunnarsson/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [22]:
for column in Y_test.columns:
    print('Feature: {}\n'.format(column))
    print(classification_report(Y_test[column],cv_Y_pred_data_frame[column]))

Feature: request

              precision    recall  f1-score   support

         0.0       0.83      0.98      0.90      4302
         1.0       0.48      0.11      0.17       940

    accuracy                           0.82      5242
   macro avg       0.66      0.54      0.54      5242
weighted avg       0.77      0.82      0.77      5242

Feature: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5218
         1.0       0.00      0.00      0.00        24

    accuracy                           0.99      5242
   macro avg       0.50      0.50      0.50      5242
weighted avg       0.99      0.99      0.99      5242

Feature: aid_related

              precision    recall  f1-score   support

         0.0       0.59      0.92      0.72      3052
         1.0       0.48      0.11      0.17      2190

    accuracy                           0.58      5242
   macro avg       0.53      0.51      0.45      5242
weighted avg      

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      4830
         1.0       0.20      0.00      0.00       412

    accuracy                           0.92      5242
   macro avg       0.56      0.50      0.48      5242
weighted avg       0.86      0.92      0.88      5242

Feature: storm

              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95      4743
         1.0       0.45      0.09      0.16       499

    accuracy                           0.90      5242
   macro avg       0.68      0.54      0.55      5242
weighted avg       0.87      0.90      0.87      5242

Feature: fire

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      5196
         1.0       0.50      0.02      0.04        46

    accuracy                           0.99      5242
   macro avg       0.75      0.51      0.52      5242
weighted avg       0.99      0.99      0.99

In [28]:
def display_results(cv, y_test, y_pred, y_pred_imp):
    labels = np.unique(y_pred_imp)
    confusion_mat = multilabel_confusion_matrix(y_test, y_pred_imp, labels=labels)
    accuracy1 = (y_pred == y_test).mean()
    accuracy2 = (y_pred_imp == y_test).mean()
    
    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy first model: {}, Accuracy improved model: {}, Difference: {}".format(accuracy1, accuracy2, accuracy2 - accuracy1))
    print("\nBest Parameters:", cv.best_params_)

In [29]:
display_results(cv, Y_test, Y_pred, cv_y_pred)

Labels: [0. 1.]
Confusion Matrix:
 [[[4195  107]
  [ 841   99]]

 [[5214    4]
  [  24    0]]]
Accuracy first model: request                   0.814575
offer                     0.995040
aid_related               0.576688
medical_help              0.918161
medical_products          0.948111
search_and_rescue         0.966807
security                  0.981877
military                  0.968142
child_alone               1.000000
water                     0.934948
food                      0.885349
shelter                   0.911675
clothing                  0.982831
money                     0.979016
missing_people            0.987409
refugees                  0.966807
death                     0.952881
other_aid                 0.865509
infrastructure_related    0.933613
transport                 0.957459
buildings                 0.948302
electricity               0.979969
tools                     0.994277
hospitals                 0.986646
shops                     0.995994
aid_cent

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I changed the machine learning algorithm from random forest to adaboost classifier which improved the accuracy from 92.9% to 93.1%. 

### 9. Export your model as a pickle file

In [77]:
# save the model to file
filename = 'model_pipeline.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.